# 구글 드라이브 연동

In [0]:
# 구글 드라이브 연동
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

# 모델 다운로드 및 PATH 설정

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import imageio

# Root directory of the project
ROOT_DIR = os.path.abspath("gdrive/My Drive/Test/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# 모델 설정 및 불러오기

In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [0]:

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# 사람 인식 및 배경 제거 후 이미지 저장

In [0]:
#Load a Video
filename = 'test.mp4'
vid = imageio.get_reader(os.path.join(IMAGE_DIR, filename),  'ffmpeg')
count = 0
write_path = IMAGE_DIR + '/'

#Detect Human and Remove Background
for image in vid.iter_data():
    #Detect
    results = model.detect([image], verbose=0)
    r = results[0]
    #Remove Background
    masked_image = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                        class_names, r['scores'], show_bbox=False, captions=False)
    #Save Image
    imageio.imwrite(write_path + str(count) + ".png",masked_image)
    count += 1
    print(count)

# 배경이 제거된 영상 생성

In [0]:
#Generate Video
import cv2

height = 1440
width = 2560
size = (width,height)

masked_images = []
read_path = IMAGE_DIR + '/'
out = cv2.VideoWriter(os.path.join(IMAGE_DIR, 'output.mp4'),cv2.VideoWriter_fourcc(*'DIVX'), 30.0, size)

for i in range(2340):
    masked_image = skimage.io.imread(read_path + str(i) + ".png")
    out.write(masked_image)
out.release()